## Model training
> In this notebook we train a model for each stock and save it to disk.

### 1) Here we import the data_manager and model modules 
> * Here we use the `'companies()'` method from and `'data_manager'` module.
> * From the `'model'` module we use the `'model_selector'` method and the `'ModelLoader'` class.

In [1]:
%load_ext autoreload
%aimport data_manager
%aimport model
%autoreload 1

from data_manager import *
from model import *

Using TensorFlow backend.


###  2) Import company list
> Here we read a csv file and import a list of company trade symbols

In [2]:
stocks = companies()
symbols = stocks['Symbol'].values.tolist()
print(symbols)

['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DWDP', 'XOM', 'WBA', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS']


###  3) RNN model training
> In this section we train a RNN model for each stock.

In the model selection [notebook](./model_selection.ipynb) we seleted a bidirectional RNN model where we would pass parameter lists and then select the model with lowest MSE test. We use that model here to train every stock.

In [3]:
print(final_model.__doc__)
print(model_selector.__doc__)


    *** final_model() ***
        RNN model with following layers:
        1) Bidirectional LSTM layer (output size based on X input sequence length)
        2) Fully connected layer (output based on input sequence length)
        3) Dropout (based on given dropout rate) 
        4) Fully connected tanh output layer of 1

    Parameters
    -----------
    X: numpy array
        input sequence data.

    y: numpy array
        target sequence data.

    learn_rate: float
        Neural network learning rate.
        
    dropout: float
        Dropout rate.

    Returns 
    -----------
    Returns compiled keras sequential model
    

    *** model_selector() ***

        Selects the best peforming (base on lowest MSE on test data) model for a given ticker based on 
        the given widow sizes, learn rates, drop out rates, and epoch counts

    Parameter
    -----------
    ticker: str
        company trade symbol.

    window_size: list
        List of sequence lengths to test.

 

In [4]:
#our model selector input variables
window_sizes = [5,7,10]
dropouts =  [0.25,0.4]
learn_rates = [0.01,0.001]
epochs = [100,200]
batch_size = 50

### Train one stock
> Here lets train the first stock in our list and save to disk.

In [5]:
result = model_selector(symbols[0], window_sizes, learn_rates, dropouts, epochs, batch_size,verbose=1)

print("\nResults : ")
print("-"*60)
print(result[0])

print(result[1])
#save trained model
ModelLoader.save(result[1]['ticker'],result[0],result[1])
print("Saved trained model for {}".format(result[1]['ticker']))

*** Best Model Selection for MMM ***

Window size: 5
------------------------------------------------------------
1     > Learn rate: 0.0100 Dropout: 0.25 Epoch: 100 Training error: 0.0112 Testing error: 0.0137
2     > Learn rate: 0.0100 Dropout: 0.25 Epoch: 200 Training error: 0.0070 Testing error: 0.0148
3     > Learn rate: 0.0100 Dropout: 0.40 Epoch: 100 Training error: 0.0086 Testing error: 0.0132
4     > Learn rate: 0.0100 Dropout: 0.40 Epoch: 200 Training error: 0.0070 Testing error: 0.0157
5     > Learn rate: 0.0010 Dropout: 0.25 Epoch: 100 Training error: 0.0104 Testing error: 0.0171
6     > Learn rate: 0.0010 Dropout: 0.25 Epoch: 200 Training error: 0.0087 Testing error: 0.0138
7     > Learn rate: 0.0010 Dropout: 0.40 Epoch: 100 Training error: 0.0107 Testing error: 0.0167
8     > Learn rate: 0.0010 Dropout: 0.40 Epoch: 200 Training error: 0.0098 Testing error: 0.0136

Window size: 7
------------------------------------------------------------
9     > Learn rate: 0.0100 Dropou

### Training remaining stocks
> * Here train the remaining stocks and save the model to directory.

In [6]:
from keras import backend as K
import tensorflow as tf

for ticker in symbols[1:]:
    #release memory
    K.clear_session()
    tf.reset_default_graph()
    
    result = model_selector(ticker, window_sizes, learn_rates, dropouts, epochs, batch_size,verbose=2)
    
    #save trained model
    ModelLoader.save(result[1]['ticker'],result[0],result[1])
    print(" ==> Saved trained model for {}".format(result[1]['ticker']))



Model selection summary for AXP with window size of 7:
------------------------------------------------------------
 ==> Learn rate: 0.0100 Dropout: 0.40 Epoch: 100 Training error: 0.0082 Testing error: 0.0219
 ==> Saved trained model for AXP

Model selection summary for AAPL with window size of 7:
------------------------------------------------------------
 ==> Learn rate: 0.0100 Dropout: 0.25 Epoch: 100 Training error: 0.0069 Testing error: 0.0301
 ==> Saved trained model for AAPL

Model selection summary for BA with window size of 7:
------------------------------------------------------------
 ==> Learn rate: 0.0010 Dropout: 0.25 Epoch: 200 Training error: 0.0036 Testing error: 0.0054
 ==> Saved trained model for BA

Model selection summary for CAT with window size of 5:
------------------------------------------------------------
 ==> Learn rate: 0.0010 Dropout: 0.25 Epoch: 200 Training error: 0.0055 Testing error: 0.0081
 ==> Saved trained model for CAT

Model selection summary